# DateTimes

### Introduction

Deciding which features should be included and focused on in our linear model is an important skill of any data scientist.  As we saw previously, if we include features which are too collinear, we will improperly measure the coefficients related to our collinear features.  In addition, feature selection and prioritizing features with feature importance will help us to understand which features to devote our attention to in terms of feature engineering and domain understanding.  Finally, limiting the number of features in our model, and identifying the most crucial features in our model will make our models, and their insights more understandable.

### Working with AirBnb

For this lesson, we'll work with [AirBnb listings in Berlin](https://www.kaggle.com/brittabettendorf/berlin-airbnb-data).

In [1]:
import pandas as pd
df = pd.read_csv('listings_summary.csv.zip')

In [2]:
df.shape

(22552, 96)

As we can see, our dataset as 22,500 rows and close to 100 features.  Our goal is to predict the price.

In [3]:
# df.columns

Some of these columns include data that we do not know how to handle - mainly textual data and images.  A lot of this data is simply not in the correct format.  Let's take a look.

In [4]:
pd.set_option('display.max_rows',100)

A lot of the data is listed as objects, which is really a catch all.

In [5]:
df.dtypes[10:15]

notes          object
transit        object
access         object
interaction    object
house_rules    object
dtype: object

In [6]:
def find_object_features(df):
    return list(df.dtypes[df.dtypes == 'object'].index)

In [7]:
len(find_object_features(df))

62

### Feature engineering

Let's try to capture as much of this object data as possible.  In this lesson, we'll start with date values.

In [8]:
def find_object_feature_values(df):
    object_features = find_object_features(df)
    return df[object_features][:2].values

In [9]:
find_object_feature_values(df)[0, :3]

array(['https://www.airbnb.com/rooms/2015', '2018-11-07',
       'Berlin-Mitte Value! Quiet courtyard/very central'], dtype=object)

We can see that the second column, `last_scraped` should be a date, but is listed as an object.  It would be nice if we could use the `to_datetime` method to put our columns in the correct format.

In [10]:
pd.to_datetime(df, infer_datetime_format=True)

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

But it doesn't work as well as we would hope. Let's write a method called `is_date` that detects if our column value is a date.

In [11]:
def contains_date(column):
    regex_string = (r'^\d{1,2}-\d{1,2}-\d{4}$|^\d{4}-\d{1,2}-\d{1,2}$' + 
'|^\d{1,2}\/\d{1,2}\/\d{4}$|^\d{4}\/\d{1,2}\/\d{1,2}$')
    return column.str.contains(regex_string).any()

The regex above is wild but it's easier to understand once we understand that each `|` means `or`.  So really, we can think of the above regex as multiple different ones.  So the first one of them is:

`^\d{1,2}-\d{1,2}-\d{4}$`

Which means start with one or two digits, then one or two digits again, and then four digits.  With each set of digits separated by a hyphen.

`11-22-2018`

So contains date tells us if any column in the above contains a date.

In [12]:
def find_date_features(df):
    series_contains_date = df.apply(contains_date)
    return series_contains_date.index[series_contains_date.values]

In [13]:
date_features = find_date_features(df)

In [14]:
date_features

Index(['last_scraped', 'host_since', 'calendar_last_scraped', 'first_review',
       'last_review'],
      dtype='object')

Now we can set those features to be date time.

In [15]:
def to_dates(df):
    date_features = find_date_features(df)
    return df[date_features].astype('datetime64[ns]')

In [24]:
df_date_features = to_dates(df)

In [25]:
df_date_features.dtypes

# last_scraped             datetime64[ns]
# host_since               datetime64[ns]
# calendar_last_scraped    datetime64[ns]
# first_review             datetime64[ns]
# last_review              datetime64[ns]

df_date_features[:1]

,last_scraped,host_since,calendar_last_scraped,first_review,last_review
0,2018-11-07,2008-08-18,2018-11-07,2016-04-11,2018-10-28


In [20]:
from date_lib import add_datepart

In [26]:
def generate_new_date_columns(dates_df):
    copied_dates_df = dates_df.copy()
    for col in copied_dates_df.columns:
        add_datepart(copied_dates_df, col)
    return copied_dates_df

In [30]:
new_date_col_df = generate_new_date_columns(df_date_features)

In [32]:
len(new_date_col_df.columns)

65

Once coercing our dates, we can merge with the original dataframe, and replace each of the old columns that should be dates.

In [73]:
def merge_dfs(original_df, new_df):
    copied_original = original_df.copy()
    date_features = find_date_features(original_df)
    copied_dropped = copied_original.drop(columns = date_features)
    copied_dropped[new_df.columns] = new_df
    return copied_dropped

In [74]:
new_df = merge_dfs(df, new_date_col_df)

We can confirm that the `new_df` has fewer `object_feature` columns than the original.

In [76]:
# new_df = new_df.drop(columns = remaining_date_features)
len(find_object_features(new_df))

57

In [77]:
len(find_object_features(df))

62

Even though our `new_df` has more features.

In [79]:
len(new_df.columns)

156

### Working with Trickier Dates

Sometimes we'll have have some features that represents a date, but is worded as text.  For example, the text '6 years ago' could be translated into a date.  Let's write a function that finds this features with this text.  And also write functions to coerce this data into the proper format.

In [50]:
def contains_time_ago(column):
    regex_string = (r'^\d.*ago$')
    return column.str.contains(regex_string).any()

In [51]:
def find_time_ago_features(df):
    series_contains_time_ago = df.apply(contains_time_ago)
    return series_contains_time_ago.index[series_contains_time_ago.values]

In [52]:
find_time_ago_features(df)

Index(['calendar_updated'], dtype='object')

In [68]:
from date_lib import get_past_date

In [69]:
calendar_updated = df.calendar_updated.apply(get_past_date).astype('datetime64[ns]')

In [70]:
calendar_updated[0:3]

0   2019-04-09
1   2019-05-21
2   2019-07-02
Name: calendar_updated, dtype: datetime64[ns]

Now we can replace our `calendar_updated` column.

### Generating New Features

There are some additional features that we did not generate.  For example, subtracting date columns from another and also subtracting dates from additional key dates like holidays.  Finally, to be even more ambitious, we could match dates to past temperatures if we believe this would help our dataset.

### Summary